In [1]:
%load_ext nb_black

import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import sklearn.metrics as metrics
import utils
from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator

import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [2]:
device = utils.get_device()
print(f"Device: {device}")

Device: cuda


<IPython.core.display.Javascript object>

In [4]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "iter1",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[0])
print(args.cov[0])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC"
experiment = f"explainer/{experiment_folder}"
iteration = "iter1"
expert_type = "explainer/accuracy"
output = "g_outputs"


# val
val_tensor_preds_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "val_tensor_preds.pt"
    )
)

val_tensor_preds_bb_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "val_tensor_preds_bb.pt"
    )
)

val_tensor_y_1 = torch.load(
    os.path.join(root, experiment, iteration, expert_type, output, "val_tensor_y.pt")
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_1.size())
print(val_tensor_preds_bb_1.size())
print(val_tensor_y_1.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([121, 2])
torch.Size([121, 2])
torch.Size([121])


<IPython.core.display.Javascript object>

In [5]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter2",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC"
experiment = f"explainer/{experiment_folder}"
iteration = "iter2"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_2.size())
print(val_tensor_preds_bb_2.size())
print(val_tensor_y_2.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([77, 2])
torch.Size([77, 2])
torch.Size([77])


<IPython.core.display.Javascript object>

In [6]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter3",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC"
experiment = f"explainer/{experiment_folder}"
iteration = "iter3"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_3.size())
print(val_tensor_preds_bb_3.size())
print(val_tensor_y_3.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([108, 2])
torch.Size([108, 2])
torch.Size([108])


<IPython.core.display.Javascript object>

In [7]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter4",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC"
experiment = f"explainer/{experiment_folder}"
iteration = "iter4"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_4.size())
print(val_tensor_preds_bb_4.size())
print(val_tensor_y_4.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([33, 2])
torch.Size([33, 2])
torch.Size([33])


<IPython.core.display.Javascript object>

In [8]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter5",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC"
experiment = f"explainer/{experiment_folder}"
iteration = "iter5"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_5.size())
print(val_tensor_preds_bb_5.size())
print(val_tensor_y_5.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([116, 2])
torch.Size([116, 2])
torch.Size([116])


<IPython.core.display.Javascript object>

In [9]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter6",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/SIIM-ISIC"
experiment = f"explainer/{experiment_folder}"
iteration = "iter6"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_6.size())
print(val_tensor_preds_bb_6.size())
print(val_tensor_y_6.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16])


<IPython.core.display.Javascript object>

In [10]:
expert_type = "explainer"
output = "g_outputs"

preds = torch.cat(
    (
        val_tensor_preds_1, val_tensor_preds_2, val_tensor_preds_3, 
        val_tensor_preds_4, val_tensor_preds_5, val_tensor_preds_6
    ), dim=0
)
gt = torch.cat(
    (
        val_tensor_y_1, val_tensor_y_2, val_tensor_y_3 ,val_tensor_y_4,
        val_tensor_y_5, val_tensor_y_6
    ), dim=0
)
bb = torch.cat(
    (
        val_tensor_preds_bb_1.cpu(), val_tensor_preds_bb_2.cpu(), val_tensor_preds_bb_3.cpu(),
        val_tensor_preds_bb_4.cpu(), val_tensor_preds_bb_5.cpu(), val_tensor_preds_bb_6.cpu()
    ),dim=0
)


<IPython.core.display.Javascript object>

In [13]:
preds_tot_np = preds.argmax(dim=1).numpy()
gt_tot_np = gt.numpy()
bb_tot_np = bb.argmax(dim=1).numpy()
bb_tot_acc = metrics.accuracy_score(gt_tot_np, bb_tot_np)
g_tot_acc = metrics.accuracy_score(gt_tot_np, preds_tot_np)
g_tot_f1 = metrics.f1_score(gt_tot_np, preds_tot_np, average="micro")
g_tot_precision = metrics.precision_score(gt_tot_np, preds_tot_np, average="micro")
g_tot_recall = metrics.recall_score(gt_tot_np, preds_tot_np, average="micro")
bb_tot_recall = metrics.recall_score(gt_tot_np, bb_tot_np, average="micro")
bb_tot_f1 = metrics.f1_score(gt_tot_np, bb_tot_np, average="micro")
g_tot_cov = preds.size(0) / 500

proba = torch.nn.Softmax(dim=1)(preds)[:, 1]
test_auroc, test_aurpc = utils.compute_AUC(gt, pred=proba)
proba_bb = torch.nn.Softmax(dim=1)(bb)[:, 1]
test_auroc_bb, test_aurpc_bb = utils.compute_AUC(gt, pred=proba_bb)

print(f"G Auroc: {test_auroc}")
print(f"BB Auroc: {test_auroc_bb}")
print(f"g_tot_acc: {g_tot_acc * 100} (%)")
print(f"bb_tot_acc: {bb_tot_acc * 100} (%)")
print(f"g_tot_f1: {g_tot_f1 * 100} (%)")
print(f"g_tot_precision: {g_tot_precision * 100} (%)")
print(f"g_tot_recall: {g_tot_recall * 100} (%)")
print(f"bb_tot_recall: {bb_tot_recall * 100} (%)")
print(f"bb_tot_f1: {bb_tot_f1 * 100} (%)")
print(f"total samples covered by g: {preds.size(0)} (out of {500})")
print(f"total coverage by g: {g_tot_cov}")


G Auroc: 0.8677986658580958
BB Auroc: 0.8800099233695353
g_tot_acc: 83.65180467091295 (%)
bb_tot_acc: 84.50106157112526 (%)
g_tot_f1: 83.65180467091295 (%)
g_tot_precision: 83.65180467091295 (%)
g_tot_recall: 83.65180467091295 (%)
bb_tot_recall: 84.50106157112526 (%)
bb_tot_f1: 84.50106157112526 (%)
total samples covered by g: 471 (out of 500)
total coverage by g: 0.942


<IPython.core.display.Javascript object>